In [7]:
import sys
import gin
import torch
from pathlib import Path

from torch import nn
import torch.optim as optim
from torchinfo import summary

In [4]:
sys.path

['/home/admindme/code/ML22_opdracht1/src',
 '/home/admindme/code/ML22_opdracht1/notebooks',
 '/home/admindme/.pyenv/versions/3.9.12/lib/python39.zip',
 '/home/admindme/.pyenv/versions/3.9.12/lib/python3.9',
 '/home/admindme/.pyenv/versions/3.9.12/lib/python3.9/lib-dynload',
 '',
 '/home/admindme/.cache/pypoetry/virtualenvs/deep-learning-mwI8DU6x-py3.9/lib/python3.9/site-packages']

In [61]:
sys.path.remove('/home/admindme/code/ML22_opdracht1/notebooks')

In [3]:
sys.path.insert(0, '/home/admindme/code/ML22_opdracht1/src')

In [71]:
sys.path.insert(0, '../src')


In [5]:
from src.models import train_model

ModuleNotFoundError: No module named 'src'

In [6]:
# Changing the CWD
os.chdir('../src')
#os.chdir('../notebooks')

In [ ]:
sys.modules


In [10]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cpu device


In [21]:
#@gin.configurable
class CNN(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            #nn.Conv2d(filter2, 32, kernel_size=kernel_size, stride=1, padding=0),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2)
        )

        self.dense = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(1152, 576),
            nn.ReLU(),

            nn.Linear(576, 288),
            nn.ReLU(),
            
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [11]:
gin.parse_config_file("model.gin")

ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])

In [22]:
# 15 = aantal classes, 3 = kernel_size, 32 = grootte filter 1, 32 = grootte  filter2

model = CNN(10, 3, 32,32).to(device)
print(model)
summary(model, input_size=(64, 1, 28, 28))

CNN(
  (convolutions): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1152, out_features=576, bias=True)
    (2): ReLU()
    (3): Linear(in_features=576, out_features=288, bias=True)
    (4): ReLU()
    (5): Linear(in_features=288, out_features=10, bias=True)
  )
)


Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [64, 10]                  --
├─Sequential: 1-1                        [64, 32, 6, 6]            --
│    └─Conv2d: 2-1                       [64, 32, 28, 28]          320
│    └─ReLU: 2-2                         [64, 32, 28, 28]          --
│    └─MaxPool2d: 2-3                    [64, 32, 14, 14]          --
│    └─Conv2d: 2-4                       [64, 32, 12, 12]          9,248
│    └─ReLU: 2-5                         [64, 32, 12, 12]          --
│    └─MaxPool2d: 2-6                    [64, 32, 6, 6]            --
├─Sequential: 1-2                        [64, 10]                  --
│    └─Flatten: 2-7                      [64, 1152]                --
│    └─Linear: 2-8                       [64, 576]                 664,128
│    └─ReLU: 2-9                         [64, 576]                 --
│    └─Linear: 2-10                      [64, 288]                 166,176
│